# Data Cleaning for Gender IDEAL

Before committing, please re-run the kernel with clear any output to avoid any merge issues with jupyter and github.

## Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
#import altair as alt

pd.set_option("max_columns", None) # show all cols
pd.set_option('max_colwidth', None) # show full width of showing cols
pd.set_option("expand_frame_repr", False) # print cols side by side as it's supposed to be

Compensation & Pay Equity: Scoring
Compensation Weight: 15

In [2]:
dfc = pd.read_csv("C:/Users/Admin/Desktop/gender-ideal-main/clean_final/compensation_clean_final.csv", index_col=0)
dfc

# Note: Gender IDEAL Assessment_Pilot file labels questions as off by 2. (begins at Q23 instead of 21).
# Referring by Python Rules for Gender IDEAL instead.
# Pilot file missing Q22:
# Do you have an organization-wide written policy that prohibits asking or otherwise collecting prior compensation data from job candidates?

,Q21_A,Q21_B,Q21_C,Q21_D,Q21_None,Q21_Other,Q22,Q23,Q24_A,Q24_B,Q24_C,Q24_D,Q24_E,Q24_F,Q24_None,Q25_A,Q25_B,Q25_C,Q25_D,Q25_E,Q25_Other,Q26,Q27_A,Q27_B,Q27_C,Q27_D,Q27_None,Q27_E,Q28,Q29_A,Q29_B,Q29_C,Q29_D,Q29_E,Q29_F,Q29_G,Q29_H,Q29_I,Q29_J,Q29_None,Q30_A,Q30_B,Q30_C,Q30_D,Q30_E,Q30_F,Q31_A,Q31_B,Q31_C,Q31_D,Q31_E,Q32,Q33,Q34_A,Q34_B,Q34_C_A,Q34_C_B,Q34_C_C,Q34_C_D,Q34_C_E,Q34_C_F,Q34_C_G,Q34_C_H,Q34_C_I,Q34_C_None,Q34_D,Q35,Q36,Q37,Organization,benchmark_group,number_of_employees,number_of_years,workforce
0,NaN,NaN,Compensation reviews are performed separately from performance reviews,"The organization performs a ""job leveling"" or compensation analysis at least every two years",NaN,NaN,Yes,In development,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,In Progress,N/A - No targets were established,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure/ responses not provided for compensation adjustments.,No evaluation conducted,Yes,NaN,Company 1X,Technology,"1000-4,999 Employees",5-14 years,At least 80% of employees are Salaried
1,NaN,NaN,NaN,NaN,None,NaN,No,No,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay 

In [3]:
w21 = 0.15
w22 = 0.10
w23 = 0.10
w26 = 0.05
w27 = 0.10
w28 = 0.05
w29 = 0.10
w30 = 0.05
w31 = 0.05
w32 = 0.10
w33 = 0.05
w36 = 0.10

w = [w21, w22, w23, w26, w27, w28, w29, w30, w31, w32, w33, w36]

comp_weight = 15

# For each Company row, generate score by weights determined above.
dfc = dfc[['Organization',
           'Q21_A', 'Q21_B', 'Q21_C', 'Q21_D', 'Q21_None', 'Q21_Other', 
           'Q22', 
           'Q23',
           'Q26',
           'Q27_A', 'Q27_B', 'Q27_C', 'Q27_D', 'Q27_None', 'Q27_E', 
           'Q28',
           'Q29_A', 'Q29_B', 'Q29_C', 'Q29_D', 'Q29_E', 'Q29_F', 'Q29_G', 'Q29_H', 'Q29_I', 'Q29_J', 'Q29_None',
           'Q30_A', 'Q30_B', 'Q30_C', 'Q30_D', 'Q30_E', 'Q30_F',
           'Q31_A', 'Q31_B', 'Q31_C', 'Q31_D', 'Q31_E',
           'Q32',
           'Q33',
           'Q36']].copy()

# Rows including 'Other' display None. Change to NaN for consistency.
dfc = dfc.replace(['None'], "NaN")

# Function to apply to each row:
def compute_score4(row):
     
    s21 = 0
    s22 = 0
    s23 = 0
    s26 = 0
    s27 = 0
    s28 = 0
    s29 = 0
    s30 = 0
    s31 = 0
    s32 = 0
    s33 = 0
    s36 = 0
    
    # Q21
    # Multi-pick, cap at 100%
    
    q21a = row['Q21_A']
    q21b = row['Q21_B']
    q21c = row['Q21_C']
    q21d = row['Q21_D']
    
    if q21a == "A publicly-stated philosophy and commitment to pay transparency":
        s21 += 0.25
    if q21b == "Transparency of pay process, including salary bands/ranges that are shared with all employees":
        s21 += 0.5
    if q21c == "Compensation reviews are performed separately from performance reviews":
        s21 += 0.5
    if q21d == "The organization performs a ""job levelling"" or compensation analysis at least every two years":
        s21 += 0.5
     
    if s21 > 1:
        s21 = 1.00
        
    # Q22
    # Single-pick
    q22 = row['Q22']
    if q22 == "In development":
        s22 = 0.5
    if q22 == "Yes":
        s22 = 1.0

    # Q23
    # Single-pick
    q23 = row['Q23']
    if q23 == "In development":
        s23 = 0.5
    if q23 == "Yes":
        s23 = 1.0
        
    if q23 != "No":
        
        # Q26
        # Single-pick
        q26 = row['Q26']
        if q26 == "1:1+ equity for men:women":
            s26 = 1.00
        if q26 == "1:0.8-0.99 equity for men:women":
            s26 = 0.5

        if (q26 == "1:0.6-0.79 equity for men:women"
            or q26 == "1:<0.6 equity for men:women"
            or q26 == "N/A"):
            s26 = 0

        # Q27
        # Multi-pick, cap at 100%
        # Q27_None and _E removed (N/A... unsure columns)
        q27a = row['Q27_A']
        q27b = row['Q27_B']
        q27c = row['Q27_C']
        q27d = row['Q27_D'] 

        if q27a == "Entry Level":
            s27 += 0.25

        if q27b == "Management/ Supervisory Level":
            s27 += 0.25

        if q27c == "Executive Management":
            s27 += 0.25

        if q27d == "C-suite Level":
            s27 += 0.25

        # Q28
        # Single-pick
        q28 = row['Q28']
        if q28 == "Yes, equity is 1:.98+ for men:women = 100%":
            s28 = 1.00

        # Q29
        # Multi-pick, cap at 100%
        q29a = row['Q29_A']
        q29b = row['Q29_B']
        q29c = row['Q29_C']
        q29d = row['Q29_D']
        q29e = row['Q29_E']
        q29h = row['Q29_H']
        q29i = row['Q29_I']

        if q29a == "Black or African American women":
            s29 += 0.5
        if q29b == "Latina women":
            s29 += 0.5
        if q29c == "Asian women":
            s29 += 0.5
        if q29d == "Other (Native American/Pacific Islander, Alaskan)":
            s29 += 0.5
        if q29e == "Mixed Race":
            s29 += 0.5
        # Q29_F: "White / Caucasian women"
        # Q29_G: "Working moms (of school-aged children)"
        if q29h == "LGTBQ women":
            s29 += 0.5
        if q29i == "Gender non-binary":
            s29 += 0.5

        if s29 > 1:
            s29 = 1.00

        # Q30
        # Multipick, cap at 100%
        q30a = row['Q30_A']
        q30b = row['Q30_B']
        q30c = row['Q30_C']
        q30d = row['Q30_D']
        q30e = row['Q30_E']
        q30f = row['Q30_F']

        if q30a != "No":
            if q30b == "With C-Suite":
                s30 += 0.25
            if q30c == "With Board/Governing Body":
                s30 += 0.25
            if q30d == "With Managers":
                s30 += 0.25
            if q30e == "With All Workers":
                s30 += 0.25
            if q30f == "Publicly":
                s30 += 0.25   
        if s30 > 1:
            s30 = 1.00

        # Q31
        # Multipick, cap at 100%
        q31a = row['Q31_A']
        q31b = row['Q31_B']
        q31c = row['Q31_C']
        q31d = row['Q31_D']
        q31e = row['Q31_E']

        if q31a != "No":
            if q31b == "With C-Suite":
                s31 += 0.2
            if q31c == "With Board/Governing Body":
                s31 += 0.2
            if q31d == "With Managers":
                s31 += 0.2
            if q31e == "With All Workers":
                s31 += 0.4

    # Q32 / Q33
    # Single-pick / Single-pick
    # Skip 33 if 32 is No
    q32 = row['Q32']
    if q32 != "No":
        
        if q32 == "In Progress":
            s32 = 0.5
        if (q32 == "Yes, shared internally with employees"
            or q32 == "Yes, shared externally as a public-facing commitment"):
            s32 = 1.00
            
        q33 = row['Q33']
        if q33 == "N/A - No targets were established":
            s33 = 0
        if q33 == "Behind goals (<66% or fewer targets met)":
            s33 = 0.33
        if (q33 == "Nearing goals (66-99% targets met)" 
            or q33 == "Goals met" 
            or q33 == "Goals exceeded"):
            s33 = 1.00
    
    
    # Q36: 
    # Single-pick
    
    q36 = row['Q36']

    if q36 != "Yes":     
        s36 = 1.0
    
    s = [s21, s22, s23, s26, s27, s28, s29, s30, s31, s32, s33, s36]
    
    return sum(map(lambda a, b: a * b, s, w))

dfc.loc[:,'comp_prelim_score'] = dfc.apply(compute_score4, axis = 1)
dfc.loc[:,'comp_weighted_score'] = dfc['comp_prelim_score'] * comp_weight

final_scores = pd.DataFrame(dfc, columns=['Organization', 'comp_weighted_score'])
final_scores

,Organization,comp_weighted_score
0,Company 1X,4.1250
1,Company 1W,0.0000
2,Company 1V,2.0625
3,Company 1U,3.3750
4,Company 1T,5.0625
5,Company 1S,1.1250
6,Company 1R,3.1875
7,Company 1Q,8.0625
8,Company 1P,8.9625
9,Company 1O,5.3625


In [4]:
#Write out to csv
final_scores.to_csv("C:/Users/Admin/Desktop/gender-ideal-main/scores/compensation_scores_clean.csv")